# HBC_TSY_ENCHANCED

# As Jobs

## Pipeline (load/transform/persist)

In [ ]:
from pathlib import Path
import tempfile

dir_base = Path(tempfile.gettempdir()) / "HBC_TASK_BASE_FOLDER"
dir_analytics = dir_base / "ANALYTICS"
dir_logging = dir_base / "LOGGING"

%cd -q ..
!python -m hbc.jobs.dispatch \
  --job-name=job_fetch_nyc_open_data_311_service_requests \
  --as-of=20091231 \
  --dir-base="{dir_base}" \
  --dir-analytics="{dir_analytics}" \
  --dir-logging="{dir_logging}" \
  --incremental=True \
  --log-level=DEBUG
%cd -q notebooks/


In [ ]:
# restore cache integrity for the last missing dates
%cd -q ..
!python -m hbc.jobs.dispatch  \
      --job-name=job_fetch_nyc_open_data_311_service_requests \
      --as-of=20091231 \
      --incremental=false \
      --log-level=INFO \
      --last-missing-dates=10
%cd -q notebooks/

## Analytics

In [ ]:
%cd -q ..
!python -m hbc.jobs.dispatch  \
      --job-name=job_analyse_nyc_open_data_311_service_requests \
      --as-of=20091231 \
      --log-level=INFO \
      --n-worst=10 \
      --n-best=10 \
      --n-days=10
%cd -q notebooks/

***

# As Library 

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

### Imports

In [2]:
import os
import sys
from pathlib import Path

In [3]:
p = str(Path.cwd().parent) # one dir up
if p not in sys.path:
    sys.path.insert(0, p)

import pandas as pd
import os
import numpy as np
import logging
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

## Api

In [4]:
from hbc import app_context, DataContainer, utils as ul
from hbc.quant.analysis import AnalyticalEngine
from hbc.quant.plots import PlotEngine

In [5]:
app_context

AppContext
as_of : 2025-12-29
dir_base: PosixPath('/var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp'),
dir_analytics: PosixPath('/var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/ANALYTICS'),
dir_logging: PosixPath('/var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/LOGS')

## Logging

In [6]:
import logging
logger = logging.getLogger()
ul.conf_log(level=logging.INFO, console=True, file=True, reset_handlers=True)

Log file: /var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/LOGS/hbc_job_generic.txt


<RootLogger root (INFO)>

## Data Container : nyc_open_data_311_customer_satisfaction_survey

In [ ]:
dc = DataContainer("nyc_open_data_311_customer_satisfaction_survey")

In [ ]:
# retrieve: query first 100 rows
dc.get()
dc.df.shape

In [ ]:
# retrieve: query distinct
dc.get(query="$apply=groupby((campaign))")
dc.df.shape

In [ ]:
# retrieve: query with filter
dc.get(query="$filter=campaign eq 'Campaign 4'")
dc.df.shape

In [ ]:
# caching: 
dc.to_cache()

In [ ]:
# from_cache: get 100 rows
dc.from_cache()
dc.df.shape

In [ ]:
# from_cache: get by filter
dc.from_cache(query="$filter=campaign eq 'Campaign 4'")
dc.df.shape

In [ ]:
# from_cache: get distinct
dc.from_cache(query="$apply=groupby((year))")
dc.df.shape

In [ ]:
# from_cache: get page 2 with page size 50
dc.from_cache(query="$top=50&$skip=50")
dc.df.shape

In [ ]:
# from_cache: get total count
dc.from_cache(query="$count=true")
dc.df.shape

## Analytics

In [ ]:
app_context.as_of  = ul.str_as_date('20091231')

In [ ]:
dc = DataContainer('nyc_open_data_311_service_requests')

In [ ]:
# query / load / validate:
dc.get(query=f"$filter=created_date eq '{ul.date_as_iso_format(app_context.as_of)}'")

In [ ]:
# persist:
dc.to_cache()

In [ ]:
# retrieve from cache for analytics:
dc.from_cache(query=f"$filter=created_date eq '{ul.date_as_iso_format(app_context.as_of)}'")

In [ ]:
df = dc.df

In [ ]:
cols = ul.cols_as_named_tuple(df)

In [ ]:
df["hbc_days_to_close"] = (
    pd.to_datetime(df[cols.closed_date])
    - pd.to_datetime(df[cols.created_date])
).dt.days.astype("Int64")
cols = ul.cols_as_named_tuple(df)

In [ ]:
m = df[cols.hbc_days_to_close] == 0
df_closed_not_same_day = df[~m]

In [ ]:
path = ul.path_to_str(
                ul.mk_dir(app_context.dir_analytics / "plots")
                / "closed_requests_by_location.html"
            )
_ = PlotEngine.plot_geo_map(
            df=df_closed_not_same_day,
            col_latitude=cols.latitude,
            col_longitude=cols.longitude,
            aggregation="count",
            round_precision=3,
            cluster=True,
            start_zoom=11,
            tiles="CartoDB positron",
            savepath= path
        )
print(path)

In [ ]:
# by agency
res = AnalyticalEngine.descriptive_stats(
    n_best=10,
    n_worst=10,
    df=df_closed_not_same_day,
    col_metric=cols.hbc_days_to_close,
    group=[
        cols.agency,
        cols.agency_name,
    ],
)


In [ ]:
res.keys()

In [ ]:
res['worst']

## Data Container : nyc_open_data_311_call_center_inquiry

In [ ]:
dc = DataContainer('nyc_open_data_311_call_center_inquiry')

In [ ]:
# load / validate:
dc.get()
dc.df.head(2)

In [ ]:
# persist
dc.to_cache()

In [ ]:
# query at loading:
dc.get(query="$filter=agency eq 'NYPD' and date eq '2014-03-27'&$top=250")
dc.df.head(2)

In [ ]:
# query at caching:
dc.from_cache(query="$filter=date eq '2014-03-27'")
dc.df.head(2)

## Data Container : nyc_open_data_311_customer_satisfaction_survey

In [ ]:
dc = DataContainer('nyc_open_data_311_customer_satisfaction_survey')

In [ ]:
# load / validate:
dc.get()
dc.df.head(2)

In [ ]:
# cache:
dc.to_cache()

In [ ]:
# query from caching:
dc.from_cache(query="$filter=answer_satisfaction eq 'Neutral'")
dc.df.head(2)

***

## `Raw DB Access`

In [ ]:
from hbc.ltp.persistence.db import SqlLiteDataBase

In [ ]:
db = SqlLiteDataBase()
print([db.run_query(f'select count(*) from {t}') for t in db.all_tables])
db.close()

## `Files/folders-based cache`

In [ ]:
from hbc.ltp.persistence.cache import Cache

In [ ]:
dc = DataContainer('nyc_open_data_311_customer_satisfaction_survey')

In [ ]:
dc.get(query='$top=10')

In [ ]:
Cache.to_cache(dc, as_of=app_context.as_of)

In [ ]:
Cache.from_cache(dc, as_of=app_context.as_of).head(2)

# Web-portal

In [13]:
path = '/Users/alexandershubert/Downloads/311_Service_Requests_for_2009_20251229.csv'

In [14]:
df = pd.read_csv(path, nrows=10)

In [20]:
sorted(df.columns.tolist())

['Address Type',
 'Agency',
 'Agency Name',
 'Borough',
 'Bridge Highway Direction',
 'Bridge Highway Name',
 'Bridge Highway Segment',
 'City',
 'Closed Date',
 'Community Board',
 'Complaint Type',
 'Created Date',
 'Cross Street 1',
 'Cross Street 2',
 'Descriptor',
 'Due Date',
 'Facility Type',
 'Ferry Direction',
 'Ferry Terminal Name',
 'Garage Lot Name',
 'Incident Address',
 'Incident Zip',
 'Intersection Street 1',
 'Intersection Street 2',
 'Landmark',
 'Latitude',
 'Location',
 'Location Type',
 'Longitude',
 'Park Borough',
 'Park Facility Name',
 'Resolution Action Updated Date',
 'Road Ramp',
 'School Address',
 'School City',
 'School Code',
 'School Name',
 'School Not Found',
 'School Number',
 'School Phone Number',
 'School Region',
 'School State',
 'School Zip',
 'School or Citywide Complaint',
 'Status',
 'Street Name',
 'Taxi Company Borough',
 'Taxi Pick Up Location',
 'Unique Key',
 'Vehicle Type',
 'X Coordinate (State Plane)',
 'Y Coordinate (State Plane)']

In [18]:
dc.df.columns.tolist()

['address_type',
 'agency',
 'agency_name',
 'borough',
 'bridge_highway_direction',
 'bridge_highway_name',
 'bridge_highway_segment',
 'city',
 'closed_date',
 'community_board',
 'complaint_type',
 'created_date',
 'cross_street_1',
 'cross_street_2',
 'descriptor',
 'due_date',
 'facility_type',
 'ferry_direction',
 'ferry_terminal_name',
 'garage_lot_name',
 'incident_address',
 'incident_zip',
 'intersection_street_1',
 'intersection_street_2',
 'landmark',
 'latitude',
 'location',
 'location_type',
 'longitude',
 'park_borough',
 'park_facility_name',
 'resolution_action_updated_date',
 'road_ramp',
 'school_address',
 'school_city',
 'school_code',
 'school_name',
 'school_not_found',
 'school_number',
 'school_or_citywide_complaint',
 'school_phone_number',
 'school_region',
 'school_state',
 'school_zip',
 'status',
 'street_name',
 'taxi_company_borough',
 'taxi_pick_up_location',
 'unique_key',
 'vehicle_type',
 'x_coordinate_state_plane_',
 'y_coordinate_state_plane_',
 '

In [7]:
!ls ../../hbc_configs/

nyc_open_data_311_call_center_inquiry.yaml
nyc_open_data_311_customer_satisfaction_survey.yaml
nyc_open_data_311_service_requests.yaml


In [6]:
dc = DataContainer('nyc_open_data_311_service_requests')

In [7]:
dc.get()

DataContainer nyc_open_data_311_service_requests missing columns; filling with None: bridge_highway_direction, bridge_highway_name, bridge_highway_segment, due_date, facility_type, ferry_direction, ferry_terminal_name, garage_lot_name, road_ramp, school_address, school_city, school_code, school_name, school_not_found, school_number, school_or_citywide_complaint, school_phone_number, school_region, school_state, school_zip, taxi_company_borough, x_coordinate_state_plane_, y_coordinate_state_plane_


In [9]:
dc.df.head()

,address_type,agency,agency_name,borough,bridge_highway_direction,bridge_highway_name,bridge_highway_segment,city,closed_date,community_board,...,y_coordinate_state_plane,open_data_channel_type,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_sbqj_enih,:@computed_region_92fq_4b7q,resolution_description,DROP_FLAG,DROP_REASON,hbc_unique_key
0,ADDRESS,DSNY,Department of Sanitation,MANHATTAN,None,None,None,NEW YORK,NaN,08 MANHATTAN,...,222665,ONLINE,23,4,11,51,NaN,False,,3815b3933f130b6af38a9eb47b6ca18190798bb5
1,ADDRESS,NYPD,New York City Police Department,BROOKLYN,None,None,None,BROOKLYN,NaN,18 BROOKLYN,...,163464,ONLINE,5,2,38,8,NaN,False,,6e0efd3a1d1705a49ee521a7849df5e2bd1681f9
2,ADDRESS,NYPD,New York City Police Department,BROOKLYN,None,None,None,BROOKLYN,NaN,11 BROOKLYN,...,162873,PHONE,1,2,37,45,NaN,False,,54d439fec934f7b2de585d6120866b743942133f
3,ADDRESS,NYPD,New York City Police Department,BRONX,None,None,None,BRONX,NaN,09 BRONX,...,237584,PHONE,58,5,26,31,NaN,False,,0d42892ecbc52d486ecc95d9a3fbf8834ae19d0c
4,ADDRESS,NYPD,New York City Police Department,MANHATTAN,None,None,None,NEW YORK,NaN,03 MANHATTAN,...,201866,MOBILE,70,4,4,50,NaN,False,,6ce1376762f19fb0c477d458c23f72fac300f6b1


In [12]:
dc.df[[k['name'] for k in dc.config['schema']]]

,address_type,agency,agency_name,borough,bridge_highway_direction,bridge_highway_name,bridge_highway_segment,city,closed_date,community_board,...,school_state,school_zip,status,street_name,taxi_company_borough,taxi_pick_up_location,unique_key,vehicle_type,x_coordinate_state_plane_,y_coordinate_state_plane_
0,ADDRESS,DSNY,Department of Sanitation,MANHATTAN,None,None,None,NEW YORK,NaN,08 MANHATTAN,...,None,None,In Progress,EAST 83 STREET,None,NaN,67261544,NaN,None,None
1,ADDRESS,NYPD,New York City Police Department,BROOKLYN,None,None,None,BROOKLYN,NaN,18 BROOKLYN,...,None,None,In Progress,EAST 53 STREET,None,NaN,67260416,NaN,None,None
2,ADDRESS,NYPD,New York City Police Department,BROOKLYN,None,None,None,BROOKLYN,NaN,11 BROOKLYN,...,None,None,In Progress,19 AVENUE,None,NaN,67259338,NaN,None,None
3,ADDRESS,NYPD,New York City Police Department,BRONX,None,None,None,BRONX,NaN,09 BRONX,...,None,None,In Progress,RANDALL AVENUE,None,NaN,67257438,NaN,None,None
4,ADDRESS,NYPD,New York City Police Department,MANHATTAN,None,None,None,NEW YORK,NaN,03 MANHATTAN,...,None,None,In Progress,SUFFOLK STREET,None,NaN,67256419,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,ADDRESS,NYPD,New York City Police Department,BROOKLYN,None,None,None,BROOKLYN,NaN,09 BROOKLYN,...,None,None,In Progress,CROWN STREET,None,NaN,67263447,NaN,None,None
96,ADDRESS,NYPD,New York City Police Department,MANHATTAN,None,None,None,NEW YORK,NaN,12 MANHATTAN,...,None,None,In Progress,WEST 211 STREET,None,NaN,67263444,NaN,None,None
97,ADDRESS,NYPD,New York City Police Department,BROOKLYN,None,None,None,BROOKLYN,NaN,05 BROOKLYN,...,None,None,In Progress,WYONA STREET,None,NaN,67257444,NaN,None,None
98,ADDRESS,NYPD,New York City Police Department,BROOKLYN,None,None,None,BROOKLYN,2025-12-22 01:48:40,17 BROOKLYN,...,None,None,Closed,EAST 95 STREET,None,NaN,67257331,Van,None,None
